In [18]:
from setup import setup_src_path
print(setup_src_path())
import data.processed as processed
import config.config as config
import utils.setup as setup
import utils.functions as fn
from importlib import reload



['/home/guest/Desktop/projects/intial-experments/domain_adaptation_project/notebooks', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages', '/home/guest/Desktop/projects/intial-experments/domain_adaptation_project/modules', '/tmp/tmpsewom7iz']


In [2]:
from datasets import load_from_disk
from torch.utils.data import DataLoader

source_data=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/source_data")
source_data_eval=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/source_data_eval")
target_data=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/target_data")
target_data_eval=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/target_data_eval")
test_target_data=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/test_target_data")
unsupervised_target=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/unsupervised_target")


source_data_loader = DataLoader(source_data, batch_size=32, shuffle=True)
target_data_loader = DataLoader(target_data, batch_size=32)
source_data_eval_loader = DataLoader(source_data_eval, batch_size=32, shuffle=True)
target_data_eval_loader = DataLoader(target_data_eval, batch_size=32)

target_test_loader = DataLoader(test_target_data, batch_size=32)

In [3]:
unsupervised_target

Dataset({
    features: ['promptID', 'pairID', 'premise', 'premise_binary_parse', 'premise_parse', 'hypothesis', 'hypothesis_binary_parse', 'hypothesis_parse', 'genre', 'label'],
    num_rows: 52350
})

In [31]:
from transformers import AutoConfig
from adapters import AutoAdapterModel

mdlcfg = AutoConfig.from_pretrained(
    config.Config.MODEL_NAME,
    num_labels=3,
)
model = AutoAdapterModel.from_pretrained(
    config.Config.MODEL_NAME,
    config=mdlcfg,
)
reload(fn)
fn.print_trainable_parameters(model)

trainable params: 66985530 || all params: 66985530 || trainable%: 100.0


In [32]:
model.delete_adapter(adapter_name)

In [33]:
from adapters import PrefixTuningConfig

adapter_name="prefix_tuning_for_task"

adapter_config = PrefixTuningConfig(flat=False)
model.add_adapter(adapter_name, config=adapter_config)
fn.print_trainable_parameters(model)

trainable params: 72130106 || all params: 72130106 || trainable%: 100.0


In [34]:
model.add_classification_head(
    adapter_name,
    num_labels=3,
  )
model.set_active_adapters(adapter_name)
model.train_adapter(adapter_name)

In [35]:
fn.print_trainable_parameters(model)
model.active_adapters

trainable params: 6360125 || all params: 72723005 || trainable%: 8.745685082732761


Stack[prefix_tuning_for_task]

In [43]:
model.adapter_summary(as_dict=True)

[{'name': 'prefix_tuning_for_task',
  'architecture': 'prefix_tuning',
  'active': True,
  '#param': 5144576,
  'train': True,
  '%param': 7.752189175635536},
 {'name': 'Full model', '#param': 66362880, '%param': 100.0, 'train': False}]

In [36]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)
print(model.active_adapters)

distilbert.transformer.layer.0.attention.prefix_tuning.pool.prefix_tunings.prefix_tuning_for_task.self.wte.weight
distilbert.transformer.layer.0.attention.prefix_tuning.pool.prefix_tunings.prefix_tuning_for_task.self.control_trans.0.weight
distilbert.transformer.layer.0.attention.prefix_tuning.pool.prefix_tunings.prefix_tuning_for_task.self.control_trans.0.bias
distilbert.transformer.layer.0.attention.prefix_tuning.pool.prefix_tunings.prefix_tuning_for_task.self.control_trans.2.weight
distilbert.transformer.layer.0.attention.prefix_tuning.pool.prefix_tunings.prefix_tuning_for_task.self.control_trans.2.bias
heads.default.0.weight
heads.default.0.bias
heads.default.2.weight
heads.default.2.bias
heads.default.3.bias
heads.prefix_tuning_for_task.1.weight
heads.prefix_tuning_for_task.1.bias
heads.prefix_tuning_for_task.4.weight
heads.prefix_tuning_for_task.4.bias
Stack[prefix_tuning_for_task]


In [38]:
# Evaluate the model on the target domain before adaptation
reload(fn)
accuracy_before, f1_before = fn.evaluate_model(model, target_test_loader)
print(f"Accuracy before adaptation: {accuracy_before}")
print(f"F1 score before adaptation: {f1_before}")

Accuracy before adaptation: 0.3355263157894737
F1 score before adaptation: 0.26695973306768234


In [39]:
reload(config)
reload(fn)
trainer = fn.train_model(model,"prefix-tuning-for-task",source_data,source_data_eval)

/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


  0%|          | 0/9380 [00:00<?, ?it/s]

{'loss': 1.0587, 'learning_rate': 0.0001, 'epoch': 0.21}


  0%|          | 0/242 [00:00<?, ?it/s]

{'eval_loss': 0.9821954369544983, 'eval_accuracy': 0.49657401422107306, 'eval_f1': 0.4598990710412013, 'eval_precision': 0.6391192815683466, 'eval_recall': 0.49657401422107306, 'eval_runtime': 11.0979, 'eval_samples_per_second': 696.976, 'eval_steps_per_second': 21.806, 'epoch': 0.21}
{'loss': 0.881, 'learning_rate': 9.436936936936938e-05, 'epoch': 0.43}


...

{'eval_loss': 0.7116572260856628, 'eval_accuracy': 0.6937297996121525, 'eval_f1': 0.6938868252662744, 'eval_precision': 0.7032836296433196, 'eval_recall': 0.6937297996121525, 'eval_runtime': 11.3342, 'eval_samples_per_second': 682.446, 'eval_steps_per_second': 21.351, 'epoch': 3.62}
{'loss': 0.6657, 'learning_rate': 4.279279279279279e-06, 'epoch': 3.84}


  0%|          | 0/242 [00:00<?, ?it/s]

{'eval_loss': 0.7062340378761292, 'eval_accuracy': 0.701874595992243, 'eval_f1': 0.7014975076969809, 'eval_precision': 0.711838121430564, 'eval_recall': 0.701874595992243, 'eval_runtime': 11.3471, 'eval_samples_per_second': 681.671, 'eval_steps_per_second': 21.327, 'epoch': 3.84}


Overwriting existing adapter 'prefix_tuning_for_task'.
Overwriting existing head 'prefix_tuning_for_task'


{'train_runtime': 1087.8126, 'train_samples_per_second': 275.831, 'train_steps_per_second': 8.623, 'train_loss': 0.7523496827082847, 'epoch': 4.0}


In [40]:
trainer.evaluate(test_target_data)

  0%|          | 0/62 [00:00<?, ?it/s]

{'eval_loss': 0.7317155599594116,
 'eval_accuracy': 0.6938259109311741,
 'eval_f1': 0.693720537101176,
 'eval_precision': 0.7113346756854846,
 'eval_recall': 0.6938259109311741,
 'eval_runtime': 2.7996,
 'eval_samples_per_second': 705.814,
 'eval_steps_per_second': 22.146,
 'epoch': 4.0}

In [41]:
accuracy_before, f1_before = fn.evaluate_model(trainer.model, target_test_loader)
print(f"Accuracy before adaptation: {accuracy_before}")
print(f"F1 score before adaptation: {f1_before}")

Accuracy before adaptation: 0.6938259109311741
F1 score before adaptation: 0.693720537101176


In [42]:
trainer.model.save_adapter(f"{config.Config.ADAPTER_SAVE_PATH}/{adapter_name}", adapter_name)